In [21]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from keras.preprocessing import image
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from tensorflow.keras import layers

In [7]:
data = r"animals10\raw-img"
class_names = sorted(os.listdir(data))
class_names
for name in class_names:
    print(name,len(os.listdir(data+ "/" +name)))

butterfly 2112
cat 1668
chicken 3098
cow 1866
dog 4863
elephant 1446
horse 2623
sheep 1820
spider 4821
squirrel 1862


In [9]:
tf.random.set_seed(42)
generator = ImageDataGenerator(rescale = 1./255.,
                              validation_split=0.2,
                              horizontal_flip=True,
                              vertical_flip=True,
                              zoom_range=0.05,
                              rotation_range=25)

In [10]:
train_data = generator.flow_from_directory( data,
                                            batch_size=32, 
                                            target_size=(224, 224), 
                                            class_mode="categorical", 
                                            shuffle=True, 
                                            subset='training',
                                            seed=42)

test_data = generator.flow_from_directory( data,
                                           batch_size=32,
                                           target_size=(224, 224),
                                           class_mode="categorical",
                                           shuffle=True,
                                           subset='validation',
                                           seed=42)

Found 20947 images belonging to 10 classes.
Found 5232 images belonging to 10 classes.


In [49]:
classes_dictionary = train_data.class_indices

In [14]:
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [22]:
feature_extractor_layer = hub.KerasLayer(efficientnet_url,
                                           trainable=False, 
                                           name='feature_extraction_layer',
                                           input_shape=(224, 224, 3))

efficientnet_model = tf.keras.Sequential([
    feature_extractor_layer, 
    layers.Dense(10, activation='softmax', name='output_layer')  
  ])

In [23]:
efficientnet_model.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [25]:
efficientnet_model_history = efficientnet_model.fit(train_data, 
                        epochs=10,
                        steps_per_epoch=len(train_data),
                        validation_data=test_data)

Epoch 1/10
655/655 [==============================] - 2003s 3s/step - loss: 0.3891 - accuracy: 0.8928 - val_loss: 0.2379 - val_accuracy: 0.9281
Epoch 2/10
655/655 [==============================] - 1764s 3s/step - loss: 0.2094 - accuracy: 0.9348 - val_loss: 0.2021 - val_accuracy: 0.9343
Epoch 3/10
655/655 [==============================] - 1680s 3s/step - loss: 0.1847 - accuracy: 0.9428 - val_loss: 0.1958 - val_accuracy: 0.9407
Epoch 4/10
655/655 [==============================] - 1513s 2s/step - loss: 0.1704 - accuracy: 0.9464 - val_loss: 0.1860 - val_accuracy: 0.9421
Epoch 5/10
655/655 [==============================] - 1865s 3s/step - loss: 0.1548 - accuracy: 0.9513 - val_loss: 0.1881 - val_accuracy: 0.9396
Epoch 6/10
655/655 [==============================] - 2162s 3s/step - loss: 0.1467 - accuracy: 0.9536 - val_loss: 0.1919 - val_accuracy: 0.9390
Epoch 7/10
655/655 [==============================] - 1914s 3s/step - loss: 0.1376 - accuracy: 0.9570 - val_loss: 0.1795 - val_accuracy:

In [34]:
efficientnet_model_history.model.save('animals_10_transferlearning_efficientnet.h5')

In [46]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import layers
import tensorflow_hub as hub

# Define the custom object scope
custom_objects = {
    'KerasLayer': hub.KerasLayer
}

# Load the model
model = load_model('animals_10_transferlearning_efficientnet.h5', custom_objects=custom_objects)


In [47]:
from PIL import Image
import numpy as np

# Load and resize the image
image = Image.open('elephant2.jpg')
image = image.resize((224, 224))

# Convert the image to a numpy array
image_array = np.array(image)

# Normalize the pixel values
image_array = image_array / 255.0

# Expand dimensions to match the model input shape
input_image = np.expand_dims(image_array, axis=0)


In [59]:
# Make the prediction
prediction = model.predict(input_image)

# Get the predicted class label
predicted_class = np.argmax(prediction)
animal_name = get_key_by_value(classes_dictionary, predicted_class)
# Print the predicted class label
print("Predicted class:", animal_name)


1/1 [==============================] - 0s 123ms/step
Predicted class: elephant


In [58]:
def get_keys_by_value(dictionary, value):
    return [key for key, val in dictionary.items() if val == value]



In [61]:
import tensorflow as tf
assert tf.__version__.startswith('2')

from mini_lucid_tf2 import objectives, transformations, render

In [74]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (K  (None, 1280)             4049564   
 erasLayer)                                                      
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 4,062,374
Trainable params: 12,810
Non-trainable params: 4,049,564
_________________________________________________________________


In [81]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Create an instance of the Inception-v3 model
inception_model = InceptionV3(weights=None, include_top=True)


In [83]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
num_classes = 10  # Replace with the number of classes in your dataset

# Remove the existing top layers
inception_model.layers.pop()
inception_model.layers.pop()

# Add new top layers for your classification task
output = inception_model.layers[-1].output
output = Dense(num_classes, activation='softmax')(output)

# Create a new model with your custom top layers
custom_inception_model = Model(inputs=inception_model.input, outputs=output)


In [84]:
custom_inception_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                           

In [86]:
custom_inception_model.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])
custom_inception_model_trained = custom_inception_model.fit(train_data, 
                        epochs=1,
                        steps_per_epoch=len(train_data),
                        validation_data=test_data)

655/655 [==============================] - ETA: 0s - loss: 2.2264 - accuracy: 0.1930 

KeyboardInterrupt: 